In [ ]:
import pytorch_lightning as pl
from modules.lifter_2d_3d.model.linear_model.lit_linear_model import LitSimpleBaselineLinear
from script.lifter_2d_3d.common.dataset import construct_drive_and_act_dataset
from modules.utils.convention import get_saved_model_path
from modules.experiments.experiment import Experiment

pl.seed_everything(1234)

constructed_loader = construct_drive_and_act_dataset()
pretrained_model_path = get_saved_model_path(
        model_name='linear_model',
        model_suffix='all_actors',
        trained_dataset_name='drive_and_act',
        trained_datasubset_name='predicted_2d',
    )
saved_model_path = get_saved_model_path(
    model_name='linear_model',
    model_suffix='all_actors',
    trained_dataset_name='drive_and_act',
    trained_datasubset_name='predicted_2d_all_actors',
    pretrained_dataset_name='synthetic_cabin_ir',
    pretrained_datasubset_name='A_Pillar_Codriver',
)
experiment = Experiment(
    LitModel=LitSimpleBaselineLinear,
    construted_loaders=constructed_loader,
    pretrained_model_path=pretrained_model_path,
    saved_model_path=saved_model_path,

)

experiment.setup()
experiment.lit_model.learning_rate = 1e-3
experiment.train()